# <span style="color:darkblue"> Lecture 12 - Replacing and recoding variables</span>

<font size = "5">

- Starting a new module on manipulating data
- We will discuss NaNs and how to clean data


# <span style="color:darkblue"> I. Import Libraries and Data </span>


<font size = "5">
Key libraries

In [22]:
import numpy as np
import pandas as pd

<font size = "5">

Read dataset on car racing circuits

- https://en.wikipedia.org/wiki/Formula_One <br>
- [See Data Source](https://www.kaggle.com/datasets/rohanrao/formula-1-world-championship-1950-2020)

In [23]:
circuits = pd.read_csv("data_raw/circuits.csv")

<font size = "5">

The dataset "codebook" is a table with ...

- Key column information
- Main things:  Field, Type, and Description

<img src="figures/codebook_circuits.png" alt="drawing" width="500"/>


In [24]:
# The codebook contains basic about the columns
# "Field" is the name given to the name of the column
# "Type"  is the variable type (with the number of characters in parenthesis):
#         integer (int)
#         string (varchar - "variable character") 
#         float (float)
# "Description" contains a label with the content of the variable

<font size = "5">

Quick Discussion:
- What does the "alt" column represent?
- What does "varchar(255)" mean?

In [25]:
# var-can have different number of characters
# 255 represents the maximum number characters it can contain

# <span style="color:darkblue"> II. NaNs </span>

<font size = "5">

- Means "Not a Number"
- Used to denote missing values

In [26]:
# "NaNs" are a special number, available in the "np" library

np.nan
#creates one element that is a missing number
#potential issue for database

nan

<font size = "5">

Operations on arrays with NaNs

In [27]:
# Create two array with and without "NaNs"
# The "np.array()" functions converts a list to an array

vec_without_nans = np.array([1,1,1])
vec_with_nans = np.array([np.nan,4,5])

# When you add the vectors
# you will produce an error on any entries with "NaNs"
print(vec_without_nans)
print(vec_with_nans)
print(vec_without_nans * vec_with_nans)
print(vec_without_nans / vec_with_nans)
print(vec_without_nans + vec_with_nans)
print(vec_without_nans - vec_with_nans)


[1 1 1]
[nan  4.  5.]
[nan  4.  5.]
[ nan 0.25 0.2 ]
[nan  5.  6.]
[nan -3. -4.]


<font size = "5">

Summary statistics with NaNs

In [28]:
# Some summary statistics will not work if there are "NaNs"
# The "np.mean()" doesn't work if the mean includes "NaNs"
print(np.mean(vec_with_nans))

# Some commands ignore the "nans"
# The "np.nanmean()" computes the mean over the numeric obvservations
print(np.nanmean(vec_with_nans))

nan
4.5


<font size = "5">

Pandas summary statistics ignore NaNs

In [29]:
# This command creates an empty dataframe
# then creates a new column called "x" and computes its mean
# Note: If a column contains missing values, then the "mean" won't be 
#       representative of the whole sample

dataset = pd.DataFrame([])
dataset["x"] = vec_with_nans
print(dataset)
print(dataset["x"].mean())
#python automatically finds the mean without the nan, but pd does not


     x
0  NaN
1  4.0
2  5.0
4.5


# <span style="color:darkblue"> II. Data Cleaning</span>

<font size = "5">

- Data collection isn't perfect!
- Need to adjust values with incorrect formatting

<font size = "5">
Get data types

In [30]:
# Produces a list with the data types of each column
# Columns that say "object" have either strings or 
# a mix of string and numeric values
# match this up with the codebook to see which columns has issues
# alt should in theory be an int...
# likely has a mistake there
circuits.dtypes

circuitId       int64
circuitRef     object
name           object
location       object
country        object
lat           float64
lng           float64
alt            object
url            object
dtype: object

<font size = "5">

Check rows with numeric values

In [31]:
# The ".str.isnumeric()" checks whether each row is numeric or now.
# Using the "." twice is an example of "piping"
# which refers to chaining two commands "str" and "isnumeric()"

circuits["alt"].str.isnumeric()

0      True
1      True
2      True
3      True
4      True
      ...  
72     True
73     True
74     True
75    False
76    False
Name: alt, Length: 77, dtype: bool

In [32]:
print(circuits["alt"][0])
print(type(circuits["alt"][0]))
#BUT
(circuits["alt"][0]).isnumeric()

10
<class 'str'>


True

<font size = "5">

Extract list of non-numeric values

In [34]:
circuits.query("alt.str.isnumeric() == False")

,circuitId,circuitRef,name,location,country,lat,lng,alt,url
22,80,vegas,Las Vegas Strip Street Circuit,Las Vegas,United States,36.1147,-115.1730,\N,https://en.wikipedia.org/wiki/Las_Vegas_Grand_...
71,73,baku,Baku City Circuit,Baku,Azerbaijan,40.3725,49.8533,-7,http://en.wikipedia.org/wiki/Baku_City_Circuit
75,78,losail,Losail International Circuit,Al Daayen,Qatar,25.4900,51.4542,\N,http://en.wikipedia.org/wiki/Losail_Internatio...
76,79,miami,Miami International Autodrome,Miami,USA,25.9581,-80.2389,\N,http://en.wikipedia.org/wiki/Miami_Internation...


In [33]:
# We can reference subattributes of columns in ".query()"
# The pd.unique() function extracts unique values from a list

subset = circuits.query("alt.str.isnumeric() == False")
list_unique = pd.unique(subset["alt"])
print(list_unique)


['\\N' '-7']


In [37]:
#different uses of .str
print(circuits["location"])
print(circuits["location"].str.upper())

0        Melbourne
1     Kuala Lumpur
2           Sakhir
3         Montmeló
4         Istanbul
          ...     
72        Portimão
73         Mugello
74          Jeddah
75       Al Daayen
76           Miami
Name: location, Length: 77, dtype: object
0        MELBOURNE
1     KUALA LUMPUR
2           SAKHIR
3         MONTMELÓ
4         ISTANBUL
          ...     
72        PORTIMÃO
73         MUGELLO
74          JEDDAH
75       AL DAAYEN
76           MIAMI
Name: location, Length: 77, dtype: object


<font size = "5">

Replace certain values

In [62]:
# "list_old" encodes values we want to change
# "list_new" encodes the values that will "replace" the old
list_old = ['\\N','-7']
list_new = [np.nan,-7]

# This command replaces the values of the "alt" column
circuits["alt"] = circuits["alt"].replace(list_old, list_new)

# Note: The option "inplace = True" permanently modifies the column
# circuits["alt"].replace(list_old, list_new, inplace = True)
print(circuits["alt"])


0     10
1     10
2     10
3     10
4     10
      ..
72    10
73    10
74    10
75    10
76    10
Name: alt, Length: 77, dtype: int64


In [61]:
#changing the type of column
print(type(circuits["alt"][0]))
pd.to_numeric(circuits["alt"])
print(type(circuits["alt"][0]))

<class 'numpy.int64'>
<class 'numpy.int64'>


In [60]:
print(type(pd.to_numeric(circuits["alt"][0])))

<class 'numpy.int64'>


In [59]:
#for everything
circuits["alt"] = pd.to_numeric(circuits["alt"])
circuits["alt"]

0     10
1     10
2     10
3     10
4     10
      ..
72    10
73    10
74    10
75    10
76    10
Name: alt, Length: 77, dtype: int64

In [58]:
circuits["alt"]

0     10
1     10
2     10
3     10
4     10
      ..
72    10
73    10
74    10
75    10
76    10
Name: alt, Length: 77, dtype: int64

<font size = "5">

Store a "cleaned" dataset

In [ ]:
# After the cleaning process is done, you may want to store the dataset again
# It's recommended to do this in a separate file from the original
# That way you can always go back to the original if you made a coding error

circuits.to_csv("data_clean/circuits.csv")


<font size = "5">
Try it yourself!

- Use ".replace()" with the "country" column
- Replace "UK" with "United Kingdom"

In [ ]:
# Write your own code


<font size = "5">
Try it yourself!

- What is the column type of "lat" or "lng"?
- Does it have any string variables?
- Can we use ```str.isnumeric()``` here?

In [48]:
# Write your own code
print(circuits["lat"])
#cannot use str.isnumeric() because it is not a string (float)

0    -37.84970
1      2.76083
2     26.03250
3     41.57000
4     40.95170
        ...   
72    37.22700
73    43.99750
74    21.63190
75    25.49000
76    25.95810
Name: lat, Length: 77, dtype: float64


<font size = "5">

# <span style="color:darkblue"> II. Recoding Numeric Variables </span>



<font size = "5">

<span style="color:red"> Controlled Pitfall </span> Computing a mean for a non-numeric column

In [ ]:
circuits["alt"].dtype

In [ ]:
# Uncomment this command to see the error
# The following error occurs because the data type
# is not numeric
# circuits["alt"].mean()
#WILL NOT WORK WHEN THE COLUMN IS ALREADY CHANGED

<font size = "5">

Convert column to numeric

In [54]:
# pd.to_numeric() converts a column to numeric
# Before you use this option, make sure to "clean" the variable
# as we did before by checking what the non-numeric values are

circuits["alt_numeric"] = pd.to_numeric(circuits["alt"])
print(circuits["alt_numeric"].mean())

10.0


<font size = "5">

Recode values based on an interval <br>

$ \qquad x_{bin} = \begin{cases} ``A" &\text{ if \quad} x_1 < x \le x_2 \\
                                  ``B" &\text{ if \quad} x_2 < x \le x_3 \end{cases} $



In [65]:
bins_x = [0,2500, 5000]
#x1, x2, x3
labels_x = ["Between 0 and 2500",
            "Between 2500 and 5000"]
#"A" and "B"

circuits["bins_alt"] = pd.cut(circuits["alt_numeric"],
                              bins = bins_x,
                              right = True,
                              labels = labels_x)

circuits["bins_alt"]
# Note: if we set bins_x = [float("-inf"),2500, float("inf")]
#       then intervals are "Less than or equal to 2500" and "Above 2500"
#       float("inf") and float("-inf") represent infinity and negative infinity
#       The "right" command indicates that the right interval is
#       "less than or equal to" or just "less than"

0     Between 0 and 2500
1     Between 0 and 2500
2     Between 0 and 2500
3     Between 0 and 2500
4     Between 0 and 2500
             ...        
72    Between 0 and 2500
73    Between 0 and 2500
74    Between 0 and 2500
75    Between 0 and 2500
76    Between 0 and 2500
Name: bins_alt, Length: 77, dtype: category
Categories (2, object): ['Between 0 and 2500' < 'Between 2500 and 5000']

<font size = "5">
Try it yourself!

- Create a new variable "hemisphere"
- Encode lattitude in (-90 and 0] as "south"
- Encode lattitude in (0 and 90] as "north"

In [ ]:
# Write your own code
